# 스포티지 뉴스 단일 크롤링

In [56]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller

# 크롬 드라이버 자동 설치 및 설정
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()

# 기간 설정
START_DATE = "2022.05.03"
END_DATE = "2023.03.03"

# URL 생성
BASE_URL = "https://search.naver.com/search.naver"
QUERY = "스포티지 유압장치 리콜"
params = (
    f"?where=news&query={QUERY}&sm=tab_opt&sort=1&photo=0&field=0&pd=3"
    f"&ds={START_DATE}&de={END_DATE}"
    f"&docid=&related=0&mynews=0&office_type=0&office_section_code=0"
    f"&news_office_checked=&nso=so%3Add%2Cp%3Afrom{START_DATE.replace('.','')}to{END_DATE.replace('.','')}"
    f"&is_sug_officeid=0&office_category=0&service_area=0"
)
url = BASE_URL + params

# 첫 페이지 로드
driver.get(url)
time.sleep(3)  # 페이지 로딩을 기다림

# 데이터를 저장할 리스트
all_data = []

def get_page_data(driver):
    """현재 페이지에서 기사 데이터를 추출하는 함수"""
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.find_all('li', class_='bx')
    data = []

    for article in articles:
        title_tag = article.find('a', class_='news_tit')
        date_tag = article.find('span', class_='info')
        link = title_tag.get('href') if title_tag else None
        
        if title_tag and date_tag:
            title = title_tag.get('title')
            date = date_tag.text.strip()
            data.append({'Title': title, 'Date': date, 'Link': link})
    
    return data

# 스크롤 및 데이터 추출
SCROLL_PAUSE_TIME = 2
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 데이터 추출
    page_data = get_page_data(driver)
    all_data.extend(page_data)
    
    # 페이지 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    
    # 새로운 스크롤 높이 계산
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# 크롤링 완료 후 브라우저 종료
driver.quit()

# 데이터를 DataFrame으로 변환
df = pd.DataFrame(all_data).drop_duplicates().reset_index(drop=True)

# DataFrame 출력
print(df.to_string())  # 전체 데이터를 출력하기 위해 to_string() 사용


                                                                                                                       Title         Date                                                                                                                              Link
0                                                                                     리콜車 왜 많지? 1년새 340만대 현대차·기아 각각 100만대 넘어  2023.01.22.                                                                      http://www.smedaily.co.kr/news/articleView.html?idxno=247633
1                                                                                           제네시스 GV70·기아 스포티지 리콜...화재 위험 보고돼  2023.01.11.                                                                     https://daily.hankooki.com/news/articleView.html?idxno=913468
2                                                                          [현대차·기아 대규모 리콜] 車 제동장치 제어 전자장비 화재 위험…“충돌 없이도 불난다”  2022.11.28.                                                        htt

In [55]:

# 데이터 저장
df.to_excel('/Users/admin/softeer/week5/스포티지뉴스.xlsx', index=False)
print("데이터가 'titles_and_dates.csv' 파일에 저장되었습니다.")


데이터가 'titles_and_dates.csv' 파일에 저장되었습니다.


In [57]:
df

,Title,Date,Link
0,리콜車 왜 많지? 1년새 340만대 현대차·기아 각각 100만대 넘어,2023.01.22.,http://www.smedaily.co.kr/news/articleView.htm...
1,제네시스 GV70·기아 스포티지 리콜...화재 위험 보고돼,2023.01.11.,https://daily.hankooki.com/news/articleView.ht...
2,[현대차·기아 대규모 리콜] 車 제동장치 제어 전자장비 화재 위험…“충돌 없이도 불난다”,2022.11.28.,http://economychosun.com/client/news/view.php?...
3,"""Possibility of a fire in conditions of no eng...",2022.11.21.,http://www.kdfnews.com/news/articleView.html?i...
4,"[단독] 현대차·기아, 화재위험 韓·美에서 수백만대 리콜",2022.11.20.,https://biz.chosun.com/industry/car/2022/11/20...
...,...,...,...
68,"""화재 위험""…기아, 美서 구형 스포티지 7만1천대 리콜",2022.10.26.,http://www.wowtv.co.kr/NewsCenter/News/Read?ar...
69,"기아, 미국서 스포티지 7만1000대 리콜… 화재 발생 가능성",2022.10.26.,https://biz.chosun.com/industry/car/2022/10/26...
70,"'차량 화재 위험'…기아, 美서 구형 스포티지 7만1000대 리콜",2022.10.26.,https://view.asiae.co.kr/article/2022102621453...
71,"기아, 美서 구형 스포티지 7만1000대 리콜…""화재 위험""",2022.10.26.,http://news.heraldcorp.com/view.php?ud=2022102...


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
import pandas as pd
import chromedriver_autoinstaller
from datetime import datetime, timedelta

# 크롬 드라이버 자동 설치 및 설정
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()

# 예시 데이터베이스 불러오기
# data = {
#     "Ministry": ["국토교통부", "환경부", "국토교통부", "국토교통부", "국토교통부", "국토교통부", "국토교통부", "국토교통부", "국토교통부", "국토교통부", "환경부", "국토교통부"],
#     "Date": ["2016-12-09", "2017-07-19", "2019-06-10", "2024-07-03", "2023-01-04", "2022-10-31", "2021-05-31", "2022-10-06", "2017-06-16", "2017-05-22", "2019-09-27", "2017-01-20"],
#     "Company": ["기아", "기아", "기아", "기아", "기아", "기아", "기아", "기아", "기아", "기아", "기아", "기아"],
#     "Model": ["구형 스포티지(KM)", "스포티지 2.0 디젤", "스포티지(QL)", "스포티지", "스포티지", "스포티지", "스포티지 등 2차종", "스포티지", "구형 쏘렌토, 카니발, 스포티지", "K5, K7, 스포티지", "스포티지", "스포티지"],
#     "Issue": ["ESC 관련 리콜", "DPF 관련 리콜", "조향장치 관련 리콜", "전자제어유압장치(HECU) 관련 리콜", "전자제어유압장치(HECU) 관련 리콜", "전자제어유압장치 관련 리콜", "전자제어유압장치 관련 리콜", "전기배선 관련 리콜", "연료공급호스 관련 리콜", "세타2엔진<GDI>관련 리콜", "매연 포집 필터(DPF) / 엔진제어장치(ECU) 관련 리콜", "리어 트레일링암 관련 리콜"]
#}

# 날짜 형식 변환
df['Date'] = pd.to_datetime(df['Date'])

# 기간 설정 함수
def get_date_range(date):
    start_date = date - timedelta(days=90)
    end_date = date + timedelta(days=90)
    return start_date.strftime("%Y.%m.%d"), end_date.strftime("%Y.%m.%d")

# 페이지 스크롤 및 데이터 추출 함수
def get_page_data(driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.find_all('li', class_='bx')
    data = []

    for article in articles:
        title_tag = article.find('a', class_='news_tit')
        date_tag = article.find('span', class_='info')
        link_tag = title_tag.get('href') if title_tag else None
        
        if title_tag and date_tag:
            title = title_tag.get('title')
            date = date_tag.text.strip()
            link = link_tag.strip()
            data.append({'Title': title, 'Date': date, 'Link': link})
    
    return data

# 각 항목에 대해 크롤링 수행
for index, row in df.iterrows():
    model = row['Model']
    issue = row['Issue']
    date = row['Date']
    
    start_date, end_date = get_date_range(date)
    query = f"{model} {issue}"
    
    # URL 생성
    base_url = "https://search.naver.com/search.naver"
    params = f"?where=news&query={query}&sm=tab_opt&sort=1&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Add%2Cp%3Afrom{start_date.replace('.','')}to{end_date.replace('.','')}&is_sug_officeid=0&office_category=0&service_area=0"
    url = base_url + params

    # 첫 페이지 로드
    driver.get(url)
    time.sleep(3)  # 페이지 로딩을 기다림

    # 데이터를 저장할 리스트
    all_data = []

    # 스크롤 및 데이터 추출
    
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 데이터 추출
        page_data = get_page_data(driver)
        all_data.extend(page_data)
        
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(2)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # 데이터를 DataFrame으로 변환
    result_df = pd.DataFrame(all_data).drop_duplicates().reset_index(drop=True)

    # 파일 이름 생성
    model_cleaned = model.replace("/", "").replace(" ", "_").replace("(", "").replace(")", "")
    issue_cleaned = issue.replace("/", "").replace(" ", "_").replace("(", "").replace(")", "")
    date_str = date.strftime("%Y%m%d")
    file_name = f"/Users/admin/softeer/project/스포티지뉴스/{model_cleaned}_{issue_cleaned}_{date_str}.xlsx"

    # DataFrame을 엑셀 파일로 저장
    result_df.to_excel(file_name, index=False)
    print(f"데이터가 '{file_name}' 파일에 저장되었습니다.")

# 크롤링 완료 후 브라우저 종료
driver.quit()
